# Batch size, Optimizers, Loss

We will study the behaviour of batch size, optimizers, and loss on a linear regression, as it makes the parameters easy to visualize.

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Fake data & basic linear regression